**The CIFAR-10 dataset**

*source*: https://www.cs.toronto.edu/~kriz/cifar.html

*description*: The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 

----------------

do in the terminal: 

1. go the folder 'data' where assigments are 
<pre> > cd ../data </pre>
2. get the data and perform train/test partition [YOU NEED ~ 300 MB] 
<pre> > chmod +x get_cifar10_dataset.sh </pre>
<pre> > ./get_cifar10_dataset.sh </pre>

In [ ]:
# Display the associated webpage in a new window
import IPython
url = 'https://www.cs.toronto.edu/~kriz/cifar.html'
iframe = '<iframe src=' + url + ' width="100%" height=500></iframe>'
IPython.display.HTML(iframe)

##### Preliminaries

In [ ]:
import numpy as np
import sys
import os
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# torch stuff
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
path_source = "../src"
sys.path.append(path_source)

In [ ]:
# local 
from datasets.ds_cifar10_load import load_CIFAR10

In [ ]:
# to make interactive plotting possible
%matplotlib inline
# for auto-reloading external modules
%load_ext autoreload
%autoreload 2

### Dataset

In [ ]:
# path to the data
path_data = "../data/cifar10"
cifar10_dir = os.path.join(path_data, 'cifar-10-batches-py')

### Classes

In [ ]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
print("There are " + str(len(classes)) + " classes:")
print(classes)

### Load data

In [ ]:
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

### Random image

In [ ]:
indx_im = np.random.randint(50000)

In [ ]:
im = X_train[indx_im,:,:,:]
plt.figure(figsize=(2, 2))
plt.imshow(im.astype('uint8'))
plt.title("[class]: %s" % classes[y_train[indx_im]]);
plt.axis('off');

### Shape, size, ...

In [ ]:
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

In [ ]:
n_train = y_train.shape[0]
n_test = y_test.shape[0]

### Look more closely 

<span style="color:red"> You are invited to play with code below  </span>  

<span style="color:red"> - change 'samples_per_class' </span>  
<span style="color:red"> - make different runs, you should see different images </span>  

In [ ]:
samples_per_class = 4
classes_to_see = np.arange(0,10)
for cls in classes_to_see:
    class_images = np.random.choice(np.arange(n_train)[y_train == cls], samples_per_class, replace=False)
    
    plt.figure(figsize=(1 * samples_per_class, 2))    
    for i, indx_im in enumerate(class_images):
        plt_idx = i + 1
        plt.subplot(1, samples_per_class, plt_idx)
        
        im = X_train[indx_im,:,:,:].astype('uint8')
        
        plt.imshow(im)
        plt.axis('off')
        if i == 0:
            plt.text(-20, 16, classes[cls])

### Transformations

It is common in Computer Vision to make different image transforms.  
*pytorch* (*torchvision* to be more precise) provides several commonly used transforms and tools to combine these transforms.

Please visit for details
https://github.com/pytorch/vision#transforms

Let's check different transformations

First we define useful functions:

In [ ]:
def tensor2im(tensor_):
    """ Bring the tensor back to image"""
    return transforms.ToPILImage()(tensor_)

def display_diff(im, im_transformed):
    """ Display a difference between original image and transformed"""
    plt.figure(figsize=(10,3))
    plt.subplot(1, 2, 1)
    plt.imshow(im)
    plt.title('original')
    plt.subplot(1, 2, 2)
    plt.imshow(im_transformed)
    plt.title('transformed') 

* pick an image

In [ ]:
indx_im = np.random.randint(50000)
im = X_train[indx_im,:,:,:] 
# 'im' now is a numpy array, will make it an image (PIL.image)
print(type(im))
im = Image.fromarray(im.astype('uint8'))
print(type(im))

* define transform itself

In [ ]:
transform = transforms.Compose([
    transforms.CenterCrop(20),
    transforms.RandomHorizontalFlip(),
])

In words we define the following transformation:  
1. crop the image at the center to have a region of the given size (20 in our case)
2. randomly horizontally flips the given image with a probability of 0.5 (probability by default)

* apply transform and check the difference

In [ ]:
# apply transform
im_transformed = transform(im)
# display results
display_diff(im, im_transformed)

<span style="color:red"> **[PROBLEM I]**: </span> 

<span style="color:red">Define the following transform </span>  
<span style="color:red">1. crop the image at a random location to have a region of the given size (24)</span>  
<span style="color:blue">use 'RandomCrop'    https://github.com/pytorch/vision#randomcropsize-padding0</span>    
<span style="color:red">2. randomly horizontally flip the given image with a probability of 0.5 </span>  
<span style="color:red">3. rescale the input image to the given 'size' (32) </span>  
<span style="color:blue"> consider to use 'Scale' https://github.com/pytorch/vision#scalesize-interpolationimagebilinear </span>  

<span style="color:red"> apply this transform and see the results </span> 

In [ ]:
#YOUR CODE HERE
transform = transforms.Compose([
    transforms.RandomCrop(24),
    transforms.RandomHorizontalFlip(),
    transforms.Scale(32)
])
# apply transform
im_transformed = transform(im)
# display results
display_diff(im, im_transformed)

Another interesting tranformation is normalization, commonly used to normalize an image, prior to training  
It operates on a Tensor rather than an image and requires two params - mean & std

In [ ]:
im_mean = [0.5, 0.5, 0.5]
im_std = [0.3, 0.3, 0.3]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=im_mean, std=im_std)
])

In [ ]:
# apply transform
tensor_transformed = transform(im)
# convert tensor to image
im_transformed = tensor2im(tensor_transformed)
# display results
display_diff(im, im_transformed)

<span style="color:red"> **[PROBLEM II]**: </span> 

<span style="color:red">Implement the function, which makes inverse transformation</span>

<span style="color:blue">look at realization of 'normalize' transform in</span>   https://github.com/pytorch/vision/blob/master/torchvision/transforms.py#L129  
<span style="color:blue"> consider to use in-place version of tensor operations as in the example of 'normalize' transform above </span>   

In [ ]:
def unnormalize(tensor, mean, std):    
    """
    Make inverse transform to 'normalize'
    
    Args:
        tensor (torch.Tensor): Tensor to unnormalize
        mean (sequence)      : Sequence of means for R, G, B channels respectively.
        std (sequence)       : Sequence of standard deviations for R, G, B channels respectively.  
        
    Returns:
        torch.Tensor: unnormalized tensor.
    """
    #YOUR CODE HERE
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor 

### Datasets and data loaders

To work with datasets pytorch provides useful abstractions, like *dataset* and *dataloder* [https://github.com/pytorch/vision#datasets].  
There are 
* prepared datasets, like MNIST, CIFAR10 and CIFAR100, COCO, etc.
* *ImageFolder* dataset, which allows you to cook dataset for yourself without much efforts.


The former is used here for Cifar10 dataset.
The later is especially useful when working with new data.

On top of the *dataset* there is *dataloader*.
The *dataloader* is used, as name suggests, to load the data.  
It does that efficiently, with multi-threading, so you should not worry about how to feed you model with the data.

Have a look at src/data_set.py where the *DataSetCifar10* is defined.  
There train & test dataloaders are bundled together, for convinence.

In [ ]:
from datasets.ds_cifar10 import DataSetCifar10

In [ ]:
path_data

In [ ]:
data_set = DataSetCifar10(path_data,
                      batch_size_train=64,
                      batch_size_val=64,
                      download=False)

To iterate over train set

In [ ]:
data_iter = iter(data_set.loader['train'])

# Mini-batch images and labels.
images, labels = data_iter.next()

print (images.size())
print (labels.size())

To iterate over test set

In [ ]:
data_iter = iter(data_set.loader['val'])

# Mini-batch images and labels.
images, labels = data_iter.next()

print (images.size())
print (labels.size())